In [2]:
#Python Standard Library
import time

#Python Packaging Index
import numpy as np
import pandas as pd
from geopy import GoogleV3
from tqdm import tqdm


In [4]:
#If the link below is outdated or not working you can download the csv from https://www.zillow.com/research/data/ using Data Type: ZHVI Single-Family Homes Time Series and Geography: Neighborhood.

df = pd.read_csv("https://files.zillowstatic.com/research/public_v2/zhvi/Neighborhood_zhvi_uc_sfr_sm_sa_mon.csv?t=1617841871")
df.insert(loc = 9, 
          column = 'Lon', 
          value = 0.0)
df.insert(loc = 9, 
          column = 'Lat', 
          value = 0.0)

In [ ]:
#Sorting dataframe by RegionID, to bring some sort of order to the dataset
df = df.sort_values(by=['RegionID'], inplace=False)
df = df.reset_index(drop=True)

#In the Event of a power failure, or Internet failure, sectioning off data into rows of 1000 and saving them to an .csv file
ranges = [x for x in range(0,df.shape[0],1000)]
ranges.append(df.shape[0])



In [ ]:

#Calls the Google Maps API through the geopy python pacakge and searches for location with the given RegionName, City, and State. Here RegionName's are synonomous with neighborhood names. Given that location, google maps api give us the latitude and longitude. 

# Even with 137 mbps connection, I was only able to get about 1.5 iterations/second. There was seemingly a rate limitation on api requests. This brings the total queue to about 3 hours to fulfill.

# According to google cloud pricing for geocoding ( https://cloud.google.com/maps-platform/pricing/ : Places->Geocoding) there is a $5 per 1000 requests pricing option. Google is generous in that upon signing up they give you $300 in credits, and then for each month after you get $200 worth of credits free. With 16096 locations to be queried, that would bring us to $80.48 for this series of quieries, about 27% of our $300 budget.

# Once you sign up gor google cloud google maps platform with $300 free credits, you will be given an API_KEY, place it below.
API_KEY = ""

start = 0
for n in range(start, len(ranges)):
    if n == 0:
        pass
    else:
        start = ranges[n-1]
        end = ranges[n]

        page = n  #MUST CHANGE WITH EACH ITERATION

        for i in tqdm(range(start,end)):
            row = df.loc[i]
            place = '{}, {}, {}, USA'.format(row['RegionName'], row['City'],row['State'])
            try:
                location = GoogleV3(api_key=API_KEY).geocode(place)
                df.at[i,'Lat'] = location.latitude
                df.at[i,'Lon'] = location.longitude
            except:
                df.at[i,'Lat'] = np.nan
                df.at[i,'Lon'] = np.nan

        part = df.loc[start:end-1]
        part.to_csv('zillow-dataset-lat-long-part{}.csv'.format(page), index = False)

In [ ]:
#There will be 3 cleaning phases:
# 1. Dealing with No Lat Lon Found
# 2. Dealing with almost Duplicates
# 3. Dealing with Visual Anomolies

# Cleaning Phase 1: Should end up with around 50 not named. These will have to be manually entered. Not bad for 60/16096.

df[df['Lat'].isna()==True]

In [12]:
# Cleaning Phase 2: because queries were searched by RegionName, City, and State. There could be duplicates that exist like:
# A. Murray Hill Neighborhood NY NY New York Newyork-Newark-Jersey City Nassau County
# B. Murray Hill Neighborhood NY NY New York Newyork-Newark-Jersey City New York County

# Where A. and B. are not duplicates as they have different county names (Nassau County & New York County). But since because we queried google based upon City, rather than County they might have the same Latitude and Longitude. 

# to find these Duplicates, switch group_type's group element. There should be around 150. 
group = ['City', 'Metro', 'CountyName']
group_type = group[0]
q = df[["State",group_type, "RegionName"]].value_counts(sort=False).reset_index()
q.columns = ["State",group_type, "RegionName", 'Counts']
display(q[q['Counts']>1])



,State,Metro,RegionName,Counts
594,AZ,Phoenix-Mesa-Scottsdale,Encanto,2
811,AZ,Phoenix-Mesa-Scottsdale,Pepperwood,2
833,AZ,Phoenix-Mesa-Scottsdale,Ray Ranch,2
946,AZ,Phoenix-Mesa-Scottsdale,Superstition Ranch,2
1196,CA,Bakersfield,Downtown,2
...,...,...,...,...
15281,WA,Seattle-Tacoma-Bellevue,Madrona,2
15347,WA,Seattle-Tacoma-Bellevue,Overlake,2
15357,WA,Seattle-Tacoma-Bellevue,Pinehurst,2
15370,WA,Seattle-Tacoma-Bellevue,Redondo,2


In [ ]:
#  Once visualized, you can continue to phase 3 of cleaning.
#  Cleaning Phase 3: go to each city and metro and look for outliers